In [1]:
import math
import random

In [2]:
class Value:
    def __init__(self, data, _prev = (), _op = ''):
        self.data = data
        self.grad = 0.0
        self._backward = None 
        self._prev = _prev
        self._op = _op
    
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), _op='+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data - other.data, (self, other), _op='-')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += -1.0 * out.grad
        out._backward = _backward
        return out
    
    def __rsub__(self, other):
        return self - other
    
    def  __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), _op='*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other

    def __pow__(self, x):
        assert isinstance(x, (int, float)), "Only supporting scalar exponentiation"
        out = Value(self.data ** x, (self, ), _op=f'**{x}')
        def _backward():
            self.grad += (x * self.data**(x-1)) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        return self * other**-1
    
    # def __rdiv__(self, other):
    #     return self * other**-1

    def exp(self): # e**x
        out = Value(math.exp(self.data), (self, ), _op='exp')
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
    
    def sigmoid(self): # Sigmoid
        x = self.data
        sig = 1 / (1 + math.exp(-x))
        #sig = (1 + (-self).exp())**-1
        out = Value(sig, (self, ), _op='sigmoid')
        def _backward():
            self.grad += (sig * (1 - sig)) * out.grad
        out._backward = _backward
        return out
    
    def silu(self): #Sigmoid Linear Unit (Swish)
        x = self.data
        sig = self.sigmoid().data
        out = Value(x * sig, (self, ), _op='silu')
        def _backward():
            self.grad += ((x * (sig * (1 - sig))) + sig) * out.grad
        out._backward = _backward
        return out

    def backprop(self): #use topological sort for all children and reverse the result
        values = []
        seen = set()
        def build_topo(self):
            stack = [self]
            while stack:
                node = stack.pop()
                if node not in seen:
                    seen.add(node)
                    stack.extend(node._prev)  # Add parents to stack
                    values.append(node)  # Add self AFTER parents
        build_topo(self)

        for v in values:
            v.grad = 0.0 # Reset gradients from previous backpropagation step
        self.grad = 1.0 # Seed gradient for this node (starting point for backprop)

        [v._backward() for v in values if v._backward is not None] #backprop from the last node (BFS)

In [34]:
class Neuron:
    def __init__(self, count: int):
        # act(w * x + b)
        self.w = [Value(random.uniform(-1, 1)) for _ in range(count)]
        self.b = Value(random.uniform(-1, 1))

    def act(self, xs: list): 
        return (sum([wi * xi for wi, xi in zip(self.w, xs)]) + self.b).silu()

    def __repr__(self): # neat printout
        return '\n'.join([f"    Weight {i+1}: {self.w[i].data:12.7f}" for i in range(len(self.w))]) + f"\n   \tBias: {self.b.data:14.7f}"
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __repr__(self):
        return '\n'.join([f"  Neuron {i+1}\n{self.neurons[i]}" for i in range(len(self.neurons))])

    def act(self, xs: list):
        return [n.act(xs) for n in self.neurons]
    
class MLP:
    def __init__(self, layers: list): # [1, 3, 5, 3, 1] -> 1,3;3,5
        self.layers = [Layer(layers[i], layers[i+1]) for i in range(len(layers)-1)]

    def __repr__(self):
        return '\n'.join([f"Layer {i+1}\n{self.layers[i]}" for i in range(len(self.layers))])
    
    def all_values(self):
        values = []
        for layer in self.layers:
            for neuron in layer.neurons:
                for wt in neuron.w:
                    values.append(wt)
                values.append(neuron.b)
        return values
    
    def __call__(self, xs: list):
        for layer in self.layers:
            pred = layer.act(xs)
        return pred
    
    def train(self, input, desired_output, epochs: int, step_size: int):
        for i in range(epochs):
            preds = [self(x) for x in input]
            n_preds = []
            for sublist in preds:
                for pred in sublist:
                    n_preds.append(pred)
            loss = sum([(ground_truth - pred)**2 for ground_truth, pred in zip(desired_output, n_preds)])
            loss.backprop()
            for val in self.all_values():
                val.data += -step_size * val.grad
        print(f"Trained for {i+1} epochs. loss: {loss.data:.5f}")

In [35]:
a = MLP([1,3,5,7,5,3,1])

In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired

a.train(xs, ys, 1000, 0.001)

Trained for 1000 epochs. loss: 1.09981
